# Test Estadísticos para Comparación de Algoritmos

## 📌 Descripción

El objetivo de este análisis es comparar distintos algoritmos de optimización (ya sean de **minimización** o **maximización**) utilizando **test estadísticos**.

Para ello, se emplean pruebas estadísticas que permiten evaluar si las diferencias en el rendimiento de los algoritmos son **significativas** o simplemente el resultado de variaciones aleatorias.

---

## 📂 Formato de Entrada de Datos

Para realizar la comparación, se deben proporcionar **ficheros CSV** con los resultados obtenidos por los algoritmos. Las características de estos archivos son:

- Se debe usar un **fichero CSV separado por cada problema** a evaluar.
- En cada fichero:
  - **Las columnas** representan los diferentes **algoritmos** a comparar.
  - **Las filas** representan **las muestras** de los resultados obtenidos en diferentes ejecuciones.
  - **La primera fila** debe contener los nombres de los algoritmos (cabecera).
- **IMPORTANTE:** Todos los ficheros CSV deben tener las **mismas cabeceras** para que los algoritmos sean comparables.




---

## 📊 Métodos de Comparación Utilizados

Para analizar las diferencias entre algoritmos, se emplean los siguientes **test estadísticos**:

### 🔹 **Mann-Whitney U Test** (Wilcoxon Rank-Sum Test)
- **Objetivo:** Comparar dos algoritmos cuando sus resultados provienen de **muestras independientes**.
- **Uso recomendado:**
  - Algoritmos probabilísticos ejecutados múltiples veces con diferentes semillas aleatorias.
  - Situaciones donde cada experimento produce una distribución distinta de valores.
- **Principio:** Evalúa si una muestra tiende a producir valores más altos o más bajos que otra, sin asumir que los datos siguen una distribución normal.

### 🔹 **Wilcoxon Signed-Rank Test**
- **Objetivo:** Comparar dos algoritmos cuando los resultados provienen de **muestras pareadas**.
- **Uso recomendado:**
  - Comparaciones en **validación cruzada k-fold** con **n repeticiones**.
  - Experimentos donde cada algoritmo es evaluado sobre el mismo conjunto de pruebas.
- **Principio:** Evalúa si hay diferencias significativas en los valores de dos muestras correlacionadas, sin asumir distribución normal.

---

## 🏆 Ranking Wins - Loss

Además de los tests estadísticos, se calcularán rankings **Wins - Loss** para medir el rendimiento de los algoritmos. Se realiza de dos maneras:

1. **Ranking Wins - Loss basado en los test estadísticos:**  
   - Se analiza cuántas veces un algoritmo es significativamente mejor que otro según los test mencionados.
  
2. **Ranking Wins - Loss basado en los resultados medios:**  
   - Se comparan los valores medios obtenidos por cada algoritmo en los experimentos, sin considerar significancia estadística.

---

## 📌 Resumen

Este análisis permite responder preguntas clave como:
✅ ¿Existen diferencias significativas entre dos algoritmos?  
✅ ¿Cuál de los algoritmos tiene mejor rendimiento en términos generales?  
✅ ¿Las diferencias observadas son consistentes o podrían deberse al azar?  

🔍 **Estos test son esenciales en la investigación y el desarrollo de algoritmos de optimización y aprendizaje automático, ya que permiten validar experimentalmente qué método es superior en una tarea específica.** 🚀


### Requerimientos

In [1]:
import numpy as np
import pandas as pd
import csv
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
from statistics import mean

### Definición del tipo de test (maximización o minimización)

In [2]:
def mejor(x,y):
    return x>y # maximización
#    return x<y # minimización

def peor(x,y):
    return x<y # maximización
#    return x>y # minimización

### Definición de la clase Ranking

In [4]:
class Ranking:
    """
    Clase Ranking para almacenar y gestionar las estadísticas de victorias y derrotas
    de un algoritmo en comparación con otros.
    """

    def __init__(self, name):
        """
        Constructor de la clase Ranking.

        Parámetros:
        - name (str): Nombre del algoritmo que se va a evaluar.

        Atributos:
        - self.name (str): Nombre del algoritmo.
        - self.wins (int): Contador de victorias del algoritmo.
        - self.losses (int): Contador de derrotas del algoritmo.
        """
        self.name = name  # Nombre del algoritmo
        self.wins = 0  # Inicializa el número de victorias en 0
        self.losses = 0  # Inicializa el número de derrotas en 0

    def __lt__(self, x):
        """
        Método especial para comparar dos objetos de tipo Ranking.

        Parámetro:
        - x (Ranking): Otro objeto Ranking con el que se comparará el actual.

        Retorna:
        - True si la diferencia (wins - losses) del objeto actual es menor que la del otro objeto.
        - False en caso contrario.

        Este método permite ordenar los rankings basándose en la diferencia entre victorias y derrotas.
        """
        return (self.wins - self.losses) < (x.wins - x.losses)

    def __str__(self):
        """
        Método especial para representar un objeto Ranking como una cadena de texto formateada.

        Retorna:
        - Una cadena con el nombre del algoritmo y sus estadísticas de victorias y derrotas,
          alineadas para una mejor visualización en tablas.

        Ejemplo de salida:
        "Algoritmo_X          12          8              4"
        """
        return f"{self.name:<15} {self.wins:>10} {self.losses:>10} {self.wins - self.losses:>15}"


### Calcula la matriz de wins - losses de las medias a partir de los scores de un problema

In [ ]:
def CalculateWinsLossesMatrixMean(scores):
    """
    Calcula la matriz de victorias y derrotas basada en la media de los resultados de cada algoritmo.

    Parámetros:
    - scores (DataFrame): Un DataFrame donde cada columna representa los resultados de un algoritmo
      en múltiples ejecuciones (filas).

    Retorna:
    - WinLossMatriz (numpy.ndarray): Matriz de comparación donde:
        - 1 indica que el algoritmo en la fila ha superado al algoritmo en la columna.
        - -1 indica que el algoritmo en la fila ha sido superado por el de la columna.
        - 0 no se usa en esta implementación ya que no hay empates explícitos.
    """

    labels = scores.columns.values  # Extrae los nombres de los algoritmos
    nScores = len(labels)  # Número de algoritmos en la comparación
    WinLossMatriz = np.zeros((nScores, nScores))  # Inicializa una matriz de ceros de tamaño nScores x nScores

    # Itera sobre cada par único de algoritmos para comparar sus medias
    for i in range(nScores - 1):
        score_i = scores.iloc[:, i].values  # Obtiene los resultados del algoritmo i

        for j in range(i + 1, nScores):
            score_j = scores.iloc[:, j].values  # Obtiene los resultados del algoritmo j

            # Compara las medias de los dos algoritmos
            if mejor(mean(score_i), mean(score_j)):
                WinLossMatriz[i, j] = 1  # Algoritmo i gana sobre j
                WinLossMatriz[j, i] = -1  # Algoritmo j pierde contra i

            if peor(mean(score_i), mean(score_j)):
                WinLossMatriz[i, j] = -1  # Algoritmo i pierde contra j
                WinLossMatriz[j, i] = 1  # Algoritmo j gana sobre i

    return WinLossMatriz


### Calcula las veces que cada algoritmo ha ganado y ha perdido a partir de la matriz de wins - losses de un problema

In [ ]:
def CalculateWinsLossesAmount(WinsLossesMatriz,labels):
    nScores = len(labels)
    WinsLossesAmount = [Ranking(scoreName) for scoreName in labels]
    for i in range(nScores-1):
        for j in range(i+1,nScores):
            if WinsLossesMatriz[i,j]==1:
                WinsLossesAmount[i].wins += 1
                WinsLossesAmount[j].losses += 1
            if WinsLossesMatriz[i,j]==-1:
                WinsLossesAmount[j].wins += 1
                WinsLossesAmount[i].losses += 1
    return WinsLossesAmount

### Acumula las cantidades de wins - losses de un problema

In [ ]:
def AddWinsLossesAmount(WinsLossesTotalAmount,WinsLossesAmount):
    for i in range(len(WinsLossesAmount)):
        WinsLossesTotalAmount[i].wins += WinsLossesAmount[i].wins
        WinsLossesTotalAmount[i].losses += WinsLossesAmount[i].losses

### Calcula la matriz de wins - losses de las diferencias estadísticamente significativas para un test estadistico dado, junto con la matriz de p-values, a partir de los scores de un problema

In [ ]:
import numpy as np
from numpy import mean  # Para calcular la media de los resultados

def CalculateWinsLossesMatrixStat(scores, stat):
    """
    Calcula la matriz de victorias y derrotas basada en diferencias estadísticamente
    significativas entre los resultados de los algoritmos evaluados.

    Parámetros:
    - scores (DataFrame): Un DataFrame donde cada columna representa los resultados de un algoritmo
      en múltiples ejecuciones (filas).
    - stat (función): Test estadístico a aplicar (por ejemplo, `scipy.stats.mannwhitneyu` o `scipy.stats.wilcoxon`).

    Retorna:
    - WinLossMatriz (numpy.ndarray): Matriz de comparación donde:
        - 1 indica que el algoritmo en la fila es significativamente mejor que el de la columna.
        - -1 indica que el algoritmo en la fila es significativamente peor que el de la columna.
        - 0 indica que no hay diferencia estadísticamente significativa.
    - pValues (numpy.ndarray): Matriz de valores p del test estadístico, indicando la significancia de la diferencia.
    """

    labels = scores.columns.values  # Extrae los nombres de los algoritmos
    nScores = len(labels)  # Número de algoritmos a comparar
    WinLossMatriz = np.zeros((nScores, nScores))  # Inicializa la matriz de wins-losses con ceros
    pValues = np.zeros((nScores, nScores))  # Inicializa la matriz de p-values con ceros

    # Itera sobre cada par único de algoritmos para aplicar el test estadístico
    for i in range(nScores - 1):
        score_i = scores.iloc[:, i].values  # Obtiene los resultados del algoritmo i

        for j in range(i + 1, nScores):
            score_j = scores.iloc[:, j].values  # Obtiene los resultados del algoritmo j

            # Verifica que los dos algoritmos no tienen exactamente los mismos resultados
            if not all(x_i == y_i for x_i, y_i in zip(score_i, score_j)):
                _, p_value = stat(score_i, score_j)  # Aplica el test estadístico
                pValues[i, j] = p_value  # Guarda el p-value en la matriz
                pValues[j, i] = p_value  # Simetría en la matriz de p-values

                # Si la diferencia es estadísticamente significativa (p-value < 0.05)
                if p_value < 0.05:
                    if mejor(mean(score_i), mean(score_j)):
                        WinLossMatriz[i, j] = 1  # Algoritmo i es significativamente mejor que j
                        WinLossMatriz[j, i] = -1  # Algoritmo j es significativamente peor que i
                    if peor(mean(score_i), mean(score_j)):
                        WinLossMatriz[i, j] = -1  # Algoritmo i es significativamente peor que j
                        WinLossMatriz[j, i] = 1  # Algoritmo j es significativamente mejor que i

    return WinLossMatriz, pValues

### Impresión de la matriz wins - losses

In [ ]:
def PrintMatriz(WinLossMatriz,labels):
    print("win: El algoritmo en la columna gana al algoritmo de la fila")
    print("loss: El algoritmo en la columna pierde frente al algoritmo de la fila")
    print("tie: El algoritmo en la columna empata con al algoritmo de la fila")
    n = len(labels)
    col_width = 10  # Ancho fijo para cada columna
    # Imprimir encabezados de columna
    print(" " * col_width, end="")
    for j in range(n):
        print(f"{labels[j]:>{col_width}}", end="")
    print()
    for i in range(n):
        print(f"{labels[i]:<{col_width}}", end="")
        for j in range(n):
            if i == j:
                print(f"{'-':>{col_width}}", end="")
            elif WinLossMatriz[j,i] == 1:
                print(f"{'win':>{col_width}}", end="")
            elif WinLossMatriz[j,i] == -1:
                print(f"{'loss':>{col_width}}", end="")
            else:
                print(f"{'tie':>{col_width}}", end="")
        print()

### Impresión  del ranking wins - losses

In [ ]:
def PrintRanking(WinLoss):
    Ranking = sorted(WinLoss, reverse=True)
    print(f"{'Ranking':<15} {'Wins':>10} {'Losses':>10} {'Wins-Losses':>15}")
    for r in Ranking:
        print(r)

### Impresión de la matriz de p-values

In [ ]:
def PrintPValuesMatriz(pValues, labels):
    n = len(labels)
    col_width = 15  # Ancho fijo para cada columna, ajusta según sea necesario
    print("p-values")
    # Imprimir encabezados de columna
    print(" " * col_width, end="")
    for j in range(n):
        print(f"{labels[j]:>{col_width}}", end="")
    print()

    # Imprimir filas con datos
    for i in range(n):
        print(f"{labels[i]:<{col_width}}", end="")
        for j in range(n):
            if i == j:
                print(f"{'-':>{col_width}}", end="")
            else:
                print(f"{pValues[i,j]:>{col_width}.8f}", end="")
        print()

### Realización de los test estadísticos e impresión de resultados

In [ ]:
fileName = ["lymphoma_11classes-results","micro-mass-results","GCM-results"] # todas deben tener las mismas cabeceras de algoritmos
# En este ejemplo son sklearn.ensemble.RandomForestClassifier, sklearn.svm.SVC y sklearn.neural_network.MLPClassifier

scores = pd.read_csv(fileName[0]+".csv") # Se coje el primero de ellos para crear los objetos de la clase Ranking
labels = scores.columns.values # Nombres de los algoritmos
WinsLossesTotalAmountMean = [Ranking(scoreName) for scoreName in labels]
WinsLossesTotalAmountRanksum = [Ranking(scoreName) for scoreName in labels]
WinsLossesTotalAmountSignedRank = [Ranking(scoreName) for scoreName in labels]

for file in fileName:

    scores = pd.read_csv(file+".csv")
    print("file = ",file)
    print(scores)
    print('\n')

    # WINS-LOSSES DE MEDIAS
    # Matriz de dos dimensiones con valores -1, 0, 1 para comparar los algoritmos en cada problema
    WinsLossesMatrizMean = CalculateWinsLossesMatrixMean(scores)
    # Lista con los objetos Ranking de cada algoritmo
    WinsLossesAmountMean = CalculateWinsLossesAmount(WinsLossesMatrizMean,labels)
    # Acumula los wins-loss de los algoritmos en todos los problemas
    AddWinsLossesAmount(WinsLossesTotalAmountMean,WinsLossesAmountMean)
    print("Mean")
    PrintMatriz(WinsLossesMatrizMean,labels)
    PrintRanking(WinsLossesAmountMean)
    print('\n')

    # WINS-LOSSES Y P-VALUES DE RANK-SUM
    # Matriz de dos dimensiones con valores -1, 0, 1 para comparar los algoritmos en cada problema
    # y matriz de dos dimensiones para los p-values
    WinsLossesMatrizRanksum, pValuesRanksum = CalculateWinsLossesMatrixStat(scores,mannwhitneyu)
    # Lista con los objetos Ranking de cada algoritmo
    WinsLossesAmountRanksum = CalculateWinsLossesAmount(WinsLossesMatrizRanksum,labels)
    # Acumula los wins-loss de los algoritmos en todos los problemas
    AddWinsLossesAmount(WinsLossesTotalAmountRanksum,WinsLossesAmountRanksum)
    print("Ranksum")
    PrintMatriz(WinsLossesMatrizRanksum,labels)
    PrintPValuesMatriz(pValuesRanksum, labels)
    PrintRanking(WinsLossesAmountRanksum)
    print('\n')

    # WINS-LOSSES Y P-VALUES DE SIGNED-RANK
    # Matriz de dos dimensiones con valores -1, 0, 1 para comparar los algoritmos en cada problema
    # y matriz de dos dimensiones para los p-values
    WinsLossesMatrizSignedRank, pValuesSignedRank = CalculateWinsLossesMatrixStat(scores,wilcoxon)
    # Lista con los objetos Ranking de cada algoritmo
    WinsLossesAmountSignedRank = CalculateWinsLossesAmount(WinsLossesMatrizRanksum,labels)
    # Acumula los wins-loss de los algoritmos en todos los problemas
    AddWinsLossesAmount(WinsLossesTotalAmountSignedRank,WinsLossesAmountSignedRank)
    print("Signed Rank")
    PrintMatriz(WinsLossesMatrizSignedRank,labels)
    PrintPValuesMatriz(pValuesSignedRank, labels)
    PrintRanking(WinsLossesAmountSignedRank)
    print('\n')


print("Total Mean")
PrintRanking(WinsLossesTotalAmountMean)
print('\n')

print("Total Ranksum")
PrintRanking(WinsLossesTotalAmountRanksum)
print('\n')

print("Total Signed Rank")
PrintRanking(WinsLossesTotalAmountSignedRank)
print('\n')


file =  lymphoma_11classes-results
          RF       SVC       MLP
0   0.900000  0.800000  0.800000
1   0.800000  0.900000  0.800000
2   0.800000  0.700000  0.900000
3   0.900000  0.800000  0.900000
4   0.800000  0.900000  1.000000
5   0.800000  0.800000  0.900000
6   0.777778  0.888889  0.888889
7   0.666667  0.666667  0.777778
8   0.888889  0.888889  0.777778
9   1.000000  0.888889  1.000000
10  0.900000  1.000000  1.000000
11  0.800000  0.700000  0.900000
12  0.900000  0.900000  0.800000
13  0.800000  0.700000  1.000000
14  0.800000  0.900000  0.900000
15  0.800000  0.800000  0.900000
16  0.666667  0.666667  0.888889
17  0.777778  0.777778  0.777778
18  0.888889  0.888889  0.777778
19  0.777778  0.888889  1.000000
20  0.900000  0.800000  0.900000
21  0.900000  0.800000  0.800000
22  0.800000  0.900000  0.900000
23  0.700000  0.700000  1.000000
24  0.800000  0.900000  0.900000
25  0.800000  0.800000  0.900000
26  0.777778  0.777778  0.888889
27  0.777778  0.888889  0.777778
28  0.77

C:\Users\Fernando\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
C:\Users\Fernando\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
C:\Users\Fernando\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn(

Los warning se deben a que los p-values son extremedamente pequeños. El cambio a una aproximación normal ocurre porque, en pruebas exactas, como la prueba de Wilcoxon o la de rangos con signo, el cálculo del valor p se basa en la distribución exacta de los datos. Este tipo de pruebas asume ciertas condiciones en la distribución de los datos para que el cálculo sea preciso. Sin embargo, cuando los valores p resultan extremadamente pequeños, puede indicar que las diferencias entre grupos son demasiado grandes, lo que altera la precisión y estabilidad del cálculo exacto.

Al tener valores p tan bajos, el test puede asumir que el cálculo exacto no es viable o que sus resultados podrían ser inestables, por lo que automáticamente opta por una aproximación normal. Esta aproximación permite estimar el valor p basándose en una distribución normal, que es robusta y adecuada para conjuntos de datos grandes o con diferencias pronunciadas, y simplifica los cálculos en esos casos extremos.

Este cambio, entonces, asegura estabilidad y eficiencia en los resultados cuando las diferencias observadas son significativas y el valor p exacto cae en el rango de números extremadamente pequeños.